The basic PMD specification essentially boils down to defining a time-invariant "basis" matrices $\{s_i\}$ and a set of interaction functions $f_i$.  The $f_i$ are functions of a control vector $\vec{c}(t)$ and a noise vector $\vec{\Gamma}(t)$. The total system hamiltonian is then defined by $H(t)=\sum f_i(\vec{c}(t),\vec{\Gamma}(t))s_i$.  Lindblad rates and operators can be defined as well to complete the master equation.

The set $\{s_i\}$ need not define an actual basis for some matrix space, but all the $s_i$ need to be of the same dimension.  It defaults to the Pauli matrices for qubit operations.  The interaction functions $f_i$ are essentially limited only by the ability to code them in Python, and can for example, include nonlinearities. 

Other than the basis and the interaction functions, the other key component is a set of parameters for each noise source in $\Gamma$, which are stored in a ``list`` of Python ``dict`` elements, one per noise source.


In [ ]:
import mezze
from mezze import PMD
from mezze.core import is_multi_qubit
from numpy import pi, sin, cos

class ExamplePMD(PMD):
    """
    This creates a PMD where there is multiplicative noise in the \sigma_x direction,
    additive noise in the \sigma_z direction, and two qubit coupling is achieved through
    \sigma_z\otimes\sigma_z, also with additive (telegraph) noise.  Specifying more than 
    two qubits generates a Hamiltonian that is defined through one and two qubit interactions.
    """
    def __init__(self, num_qubits=1):
        control_dim = 3
        noise_dim = 2
        num_lindblad = 1
        
        if is_multi_qubit(num_qubits):
            #Number of interaction control dimensions and additional noises
            two_control_dim = 1
            two_noise_dim = 1
            PMD.__init__(self, control_dim, noise_dim, num_lindblad, num_qubits, two_control_dim, two_noise_dim)
        else:
            PMD.__init__(self, control_dim, noise_dim, num_lindblad, num_qubits)        
 

        #The first index "[0]" is legacy, but the second "[1]" and "[3]" correspond
        #to \sigma_x and \sigma_z respectively.  
        #Note the factor of 1/2 that is basis dependent
        self.single_qubit_control[0][1] = lambda c, gamma: (1+gamma[0])*c[0]*cos(c[1])/2.0
        self.single_qubit_control[0][2] = lambda c, gamma: (1+gamma[0])*c[0]*sin(c[1])/2.0
        
        #If desired, could split additive noise term into single_qubit_noise[3]
        self.single_qubit_control[0][3] = lambda c, gamma: (gamma[1]+c[2])/2.0
        
        #noise_hints contains the relevant parameters for each noise in the PMD
        self.noise_hints[0]={'type':'pink','amp':1.0e-5,'omega_min':2*pi, 'omega_max':2*pi*1e6, 'ctrl': True}
        self.noise_hints[1]={'type':'exp','amp':.25, 'corr_time': 1.0, 'ctrl': False}
        
        #Some made up loss terms, can add more in list
        #Are kronned up for multi-qubit case
        self.lindblad_rates = [.25]# T1 = 4 sec
        self.linblad_operators = [[0.0,0.5,0.5*1j,0]]
        
        #The only "relevant" remaining term is self.xtalk, a matrix of crosstalk coefficients
        
        #Terms that can be defined but are essentially ignored are:
        #1. Matrices of (cross-)correlations and (cross-)spectral densities
        #2. Control constraint functions for one and two qubit controls (c_const and two_c_const)
        #3. Default self.basis is Pauli matrices and can be overwritten to e.g., qudit or multi-qudit
        
        if is_multi_qubit(num_qubits):

            #For legacy QCS reasons, need to repeat single qubit noise terms
            self.noise_hints[2]={'type':'pink','amp':1.0e-5,'omega_min':2*pi, 'omega_max':2*pi*1e6, 'ctrl': True}
            self.noise_hints[3]={'type':'exp','amp':.25, 'corr_time': 1.0, 'ctrl': False}
                                                            
            #Two qubit control defined as \sigma_z\otimes\sigma_z term
            self.two_qubit_control[0][3][3] = lambda c, gamma: (1+gamma[0])*c[0]/2.0
        
            #Two qubit noise parameters
            self.noise_hints[4]={'type':'telegraph', 'vals':[-.01,.01], 'taus':[1,1], 'ctrl': False}
            
            #If desired could define two_lindblad_rates and two_lindblad_operators


In [ ]:
import numpy as np

#Duration and number of samples for control pulses
gate_duration = 1.0
n_steps = 1000

#Identity fields for single qubit gates
zero_field = mezze.get_zero_control(n_steps)

#Pi-pulse in sigma_{zz} direction
zz_field = mezze.get_pi_pulse_control(n_steps, gate_duration)

#[Amp1, Phase1, Z1, Amp2, Phase2, Z2, ZZ]
ctrls = np.transpose(np.array([zero_field, zero_field, zero_field, zero_field, zero_field, zero_field, zz_field]))


config = mezze.SimulationConfig(time_length=gate_duration, num_steps=n_steps)
config.num_runs = 1

pmd = ExamplePMD(2)

ham_func = mezze.HamiltonianFunction(pmd)
ham = mezze.PrecomputedHamiltonian(pmd, ctrls, config, ham_func.get_function())
sim = mezze.Simulation(config, pmd, ham)

report = sim.run()

liou = report.channel.liouvillian()